# Task 3: RAG Pipeline Qualitative Evaluation

**Objective:** 
Run a set of representative questions through the RAG pipeline.
We will inspect:
1. The generated answer.
2. **Crucially:** The retrieved source documents (to ensure the AI isn't hallucinating).

Finally, we will export these results to a CSV so you can add your "Quality Score" and "Comments" for the final report.

In [1]:
import sys
import os
import pandas as pd
from dotenv import load_dotenv

# Add the scripts directory to the system path so we can import CreditRAG
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [19]:
# Import your pipeline class
from scripts.rag_pipeline import CreditRAG
# Load env vars (for HF Token)
load_dotenv()

True

## 1. Initialize the RAG System
This loads the FAISS index and connects to the Hugging Face API.

In [20]:
print("Initializing RAG System...")
try:
    rag_system = CreditRAG(vector_store_path="../vector_store/full_faiss_index")
    print("✅ System initialized successfully.")
except Exception as e:
    print(f"❌ Initialization failed: {e}")
    print("Check your API Token and Vector Store path.")

INFO:scripts.rag_pipeline:Loading Embedding Model...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Initializing RAG System...


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:scripts.rag_pipeline:Loading Vector Store from ../vector_store/full_faiss_index...
INFO:scripts.rag_pipeline:Vector Store Loaded Successfully.
INFO:scripts.rag_pipeline:Initializing LLM Endpoint...


✅ System initialized successfully.


## 2. Define Test Questions
These 7 questions cover the different products (Cards, Loans, Money Transfers) and edge cases.

In [21]:
test_questions = [
    # 1. Credit Card (General Issue)
    "What are the common complaints about credit card late fees?",
    
    # 2. Money Transfer (Specific Pain Point)
    "Why are my money transfers getting blocked or cancelled?",
    
    # 3. Loans (Process Issue)
    "What issues do customers report regarding loan modification applications?",
    
    # 4. Checking/Savings (Fees)
    "Why was I charged an overdraft fee if I didn't opt-in?",
    
    # 5. Customer Service (Cross-cutting)
    "How do customers describe the behavior of debt collectors?",
    
    # 6. Specific Company (Testing retrieval specificity)
    "What are the complaints specifically about Citibank's customer service?",
    
    # 7. Hallucination Check (Information likely not in dataset)
    "How can I invest in cryptocurrency using this platform?"
]


## 3. Execution Loop
We will run each question and print the sources clearly.

In [22]:
results_data = []

print(f"Running evaluation on {len(test_questions)} questions...\n")

for i, question in enumerate(test_questions):
    print(f"🔹 Question {i+1}: {question}")
    
    try:
        # Run the pipeline
        response = rag_system.answer_question(question)
        
        # Extract data
        answer = response['answer']
        docs = response['source_documents']
        
        print(f"🗣️ Answer: {answer}\n")
        
        # --- PRINT SOURCES CLEARLY ---
        print("🔍 Retrieved Sources:")
        formatted_sources = []
        
        for j, doc in enumerate(docs):
            # content preview (first 200 chars)
            preview = doc.page_content.replace("\n", " ")[:200]
            product = doc.metadata.get('product', 'Unknown Product')
            source_str = f"[{j+1}] ({product}): {preview}..."
            
            print(f"   {source_str}")
            formatted_sources.append(source_str)
        
        print("\n" + "="*80 + "\n")
        
        # Save to list for DataFrame
        results_data.append({
            "Question": question,
            "Generated Answer": answer,
            # Join sources with newlines for the CSV/Table
            "Retrieved Sources": "\n\n".join(formatted_sources), 
            "Quality Score (1-5)": "", # Placeholder for manual entry
            "Comments": ""             # Placeholder for manual entry
        })
        
    except Exception as e:
        print(f"❌ Error processing question '{question}': {e}")
        # If you get the 'task not supported' error here, refer to the Zephyr fix.

Running evaluation on 7 questions...

🔹 Question 1: What are the common complaints about credit card late fees?
🗣️ Answer: The common complaints about credit card late fees include: excessive and unfair fees, accumulation of interest on fraud charges that have not been removed, and blocking access to make payments. Consumers feel that these practices heavily favor the credit card companies and are taking advantage of the situation during difficult financial times. Some consumers have also reported receiving late charges and interest on supposed purchases they did not make.

🔍 Retrieved Sources:
   [1] (Credit card): i 've been issued a number of late fee that are unfair and excessive. these fees violate federal regulations put in place to prevent credit card companies from taking advantage of their customers....
   [2] (Credit card): e of consumers with late fees and damaging credit reports....
   [3] (Credit card): t or late fees since i pay my credit card in full. he assured me it wi

## 4. Generate Evaluation Table
We convert the results into a Pandas DataFrame and display it.

In [23]:
df_eval = pd.DataFrame(results_data)
# Set pandas to show full text content so we can read it in the notebook
pd.set_option('display.max_colwidth', None)
# Display the first few rows
display(df_eval)

,Question,Generated Answer,Retrieved Sources,Quality Score (1-5),Comments
0,What are the common complaints about credit card late fees?,"The common complaints about credit card late fees include: excessive and unfair fees, accumulation of interest on fraud charges that have not been removed, and blocking access to make payments. Consumers feel that these practices heavily favor the credit card companies and are taking advantage of the situation during difficult financial times. Some consumers have also reported receiving late charges and interest on supposed purchases they did not make.","[1] (Credit card): i 've been issued a number of late fee that are unfair and excessive. these fees violate federal regulations put in place to prevent credit card companies from taking advantage of their customers....\n\n[2] (Credit card): e of consumers with late fees and damaging credit reports....\n\n[3] (Credit card): t or late fees since i pay my credit card in full. he assured me it will not be an issue. i am filing this complaint because the method in which they handle this, refusing to remove fraud charges, all...\n\n[4] (Credit card or prepaid card): these past few months have been very difficult for everyone. despite the fact that people are doing all they can to pay their monthly bills, credit card companies are taking advantage of the situation...\n\n[5] (Credit card or prepaid card): ich i have no need, did not order, did not receive, do not want and will not pay for. now, i get statements every month adding late charges and interest on a supposed purchase that neither i nor my wi...",,
1,Why are my money transfers getting blocked or cancelled?,"I don't have enough information in the provided complaints to determine the exact reason for the money transfers getting blocked or cancelled. However, it seems that there may be security concerns or issues with the recipient's bank account. The context mentions that the recipient account was blocked in one instance, and the transfers were cancelled without explanation from the bank or the transfer service. The customer was advised to try changing the recipient's bank details or contacting the recipient to resolve the issue.","[1] (Checking or savings account): ty and in 1-3 days the transfer should be through. each and every time, my transfer is being blocked for no reason that they can explain other than i need to identify myself for security reasons. in t...\n\n[2] (Checking or savings account): that the transactions being blocked were routine. i have been making the same transfers every month for the last 20 years, both from the and from ....\n\n[3] (Money transfer, virtual currency, or money service): holding a different currency, or blocked from receiving money completely. because of that, your money was sent back to us. to fix your transfer, you can either change your recipients bank details and ...\n\n[4] (Money transfer, virtual currency, or money service): i had two money transfers cancelled for unknown reasons and never had this problem with this service but customer services was clueless!!!...\n\n[5] (Checking or savings account): telling me all went through when i called to confirm the next day. i'm now a week with out access to my funds, all the bank representatives tell me different things on why the transfers apparently can...",,
2,What issues do customers report regarding loan modification applications?,"Customers report issues regarding lack of transparency and potential misrepresentation during the loan modification process. Specifically, they express concerns about the lack of clear disclosure on practices related to payment modifications, interest reimbursements, and the impact of consumer grievances on subsequent loan applications. Some customers also question the precision of credit score reporting and its role in loan approval determination. However, the context provided does not mention any complaints directly related to loan modification applications that the cu

## 5. Export for Report
We save this to a CSV. You can open this CSV in Excel/Sheets to fill in your Scores and Comments, then copy it into your final report.

In [25]:
output_path = "../data/processed/rag_evaluation_results.csv"
df_eval.to_csv(output_path, index=False)
print(f"✅ Results saved to {output_path}")

✅ Results saved to ../data/processed/rag_evaluation_results.csv


In [4]:
pd.set_option('display.max_colwidth', None)
pd.read_csv('../data/processed/rag_evaluation_results.csv')

,Question,Generated Answer,Retrieved Sources,Quality Score (1-5),Comments
0,What are the common complaints about credit card late fees?,"The common complaints about credit card late fees include: excessive and unfair fees, accumulation of interest on fraud charges that have not been removed, and blocking access to make payments. Consumers feel that these practices heavily favor the credit card companies and are taking advantage of the situation during difficult financial times. Some consumers have also reported receiving late charges and interest on supposed purchases they did not make.","[1] (Credit card): i 've been issued a number of late fee that are unfair and excessive. these fees violate federal regulations put in place to prevent credit card companies from taking advantage of their customers....\n\n[2] (Credit card): e of consumers with late fees and damaging credit reports....\n\n[3] (Credit card): t or late fees since i pay my credit card in full. he assured me it will not be an issue. i am filing this complaint because the method in which they handle this, refusing to remove fraud charges, all...\n\n[4] (Credit card or prepaid card): these past few months have been very difficult for everyone. despite the fact that people are doing all they can to pay their monthly bills, credit card companies are taking advantage of the situation...\n\n[5] (Credit card or prepaid card): ich i have no need, did not order, did not receive, do not want and will not pay for. now, i get statements every month adding late charges and interest on a supposed purchase that neither i nor my wi...",5,"Excellent. High-quality retrieval of specific grievances. The answer synthesizes distinct points (fraud, excessive amounts, timing) perfectly."
1,Why are my money transfers getting blocked or cancelled?,"I don't have enough information in the provided complaints to determine the exact reason for the money transfers getting blocked or cancelled. However, it seems that there may be security concerns or issues with the recipient's bank account. The context mentions that the recipient account was blocked in one instance, and the transfers were cancelled without explanation from the bank or the transfer service. The customer was advised to try changing the recipient's bank details or contacting the recipient to resolve the issue.","[1] (Checking or savings account): ty and in 1-3 days the transfer should be through. each and every time, my transfer is being blocked for no reason that they can explain other than i need to identify myself for security reasons. in t...\n\n[2] (Checking or savings account): that the transactions being blocked were routine. i have been making the same transfers every month for the last 20 years, both from the and from ....\n\n[3] (Money transfer, virtual currency, or money service): holding a different currency, or blocked from receiving money completely. because of that, your money was sent back to us. to fix your transfer, you can either change your recipients bank details and ...\n\n[4] (Money transfer, virtual currency, or money service): i had two money transfers cancelled for unknown reasons and never had this problem with this service but customer services was clueless!!!...\n\n[5] (Checking or savings account): telling me all went through when i called to confirm the next day. i'm now a week with out access to my funds, all the bank representatives tell me different things on why the transfers apparently can...",4,"Good handling of ambiguity. The sources mostly say the reason was ""unknown"" or ""unexplained."" The model correctly reflected this uncertainty rather than hallucinating a specific bank policy."
2,What issues do customers report regarding loan modification applications?,"Customers report issues regarding lack of transparency and potential misrepresentation during the loan modification process. Specifically, they express concerns about the lack of clear disclosure on practices related to payment modific